In [1]:
%cd ~/SSMuLA

/disk2/fli/SSMuLA


In [2]:
%load_ext autoreload
%autoreload 2
%load_ext blackcellmagic

In [7]:
import pandas as pd
import numpy as np
import os
import shutil
from glob import glob

In [5]:
glob("/disk2/fli/SSMuLA/results/mlde/saved/ev_score/none-all/scale2max/DHFR/esm2*")

['/disk2/fli/SSMuLA/results/mlde/saved/ev_score/none-all/scale2max/DHFR/esm2_t33_650M_UR50D-flatten_site|esm2_t33_650M_UR50D-mean_all|esm2_t33_650M_UR50D-mean_site_boosting|ridge_sample384_top384.npy',
 '/disk2/fli/SSMuLA/results/mlde/saved/ev_score/none-all/scale2max/DHFR/esm2_t33_650M_UR50D-flatten_site|esm2_t33_650M_UR50D-mean_all|esm2_t33_650M_UR50D-mean_site_boosting|ridge_sample576_top96.npy',
 '/disk2/fli/SSMuLA/results/mlde/saved/ev_score/none-all/scale2max/DHFR/esm2_t33_650M_UR50D-flatten_site|esm2_t33_650M_UR50D-mean_all|esm2_t33_650M_UR50D-mean_site_boosting|ridge_sample576_top384.npy',
 '/disk2/fli/SSMuLA/results/mlde/saved/ev_score/none-all/scale2max/DHFR/esm2_t33_650M_UR50D-flatten_site|esm2_t33_650M_UR50D-mean_all|esm2_t33_650M_UR50D-mean_site_boosting|ridge_sample384_top96.npy']

In [6]:
glob("/disk2/fli/SSMuLA/results/mlde_emb/saved/ev_score/none-all/scale2max/DHFR/*")

['/disk2/fli/SSMuLA/results/mlde_emb/saved/ev_score/none-all/scale2max/DHFR/esm2_t33_650M_UR50D-flatten_site|esm2_t33_650M_UR50D-mean_all|esm2_t33_650M_UR50D-mean_site_boosting|ridge_sample24|48_top96.npy',
 '/disk2/fli/SSMuLA/results/mlde_emb/saved/ev_score/none-all/scale2max/DHFR/esm2_t33_650M_UR50D-flatten_site|esm2_t33_650M_UR50D-mean_all|esm2_t33_650M_UR50D-mean_site_boosting|ridge_sample24|48_top384.npy']

In [ ]:
for s in glob("/disk2/fli/SSMuLA/results/mlde/saved/ev_score/none-all/scale2max/DHFR/esm2*")

destination_path = os.path.join(destination_dir, os.path.basename(source_path))


In [8]:
glob("/disk2/fli/SSMuLA/results/mlde_ft/saved/esm_score/none-all/scale2max/DHFR/*")

['/disk2/fli/SSMuLA/results/mlde_ft/saved/esm_score/none-all/scale2max/DHFR/one-hot_boosting|ridge_sample96|384_top384.npy',
 '/disk2/fli/SSMuLA/results/mlde_ft/saved/esm_score/none-all/scale2max/DHFR/one-hot_boosting|ridge_sample96|384_top96.npy']